In [1]:
import requests
import time
import json

import re
from html import unescape

headers = {"user-agent" : "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}

def download(method, url, param=None, data=None, timeout=1 , maxretires=3):
    try : 
        resp = requests.request(method, url, params=param, data=data, headers=headers)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e :
        if 500 <= e.response.status_code < 600 and maxretires > 0:
            print(maxretires)
            time.sleep(timeout)
            resp = download(method, url, param, data, timeout, maxretires-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

# beautifulSoup : Parsing library
!pip install beautifulsoup4
from bs4 import BeautifulSoup

In [ ]:
import sqlite3
con = sqlite3.connect("bot.db")
cur = con.cursor()

# Scraping

    내가 결정할 수 있다. 얼만큼 가져올건지. (네이버 첫페이지나... 내가 원하는 만큼. 규모가 적다.)
    내가 원하는 부분만 쓰고 버려버린다.
    특정 사이트에 가서 내가 원하는 부분만 긁어온다.
    큐를 이용해 스케줄러를 대신했다.

- http://validator.w3.org/unicorn/?ucn_lang=ko
- 제목, 추천수, 조회수 를 페이지 게시물만큼 긁을 예정.

In [69]:
url = "http://www.ppomppu.co.kr/zboard/zboard.php?"

param = {
    "id":"ppomppu",
    "page":1
}
html = download("get", url, param)
dom = BeautifulSoup(html.text, "lxml")

In [70]:
[(_.text) for _ in dom.select("#revolution_main_table .list_title")]

['[G마켓] 햇반컵반 버터장조림비빔밥 216g 6개 (15,900원 /무배)',
 '[G마켓] LG 휘센 제습기 DQ168PBE (368,470원/ 무료배송)',
 '[11번가] LG 공식판매점 디오스 인덕션 전기레인지 BEF3G 3구 (1,490,000/무료)',
 '[위메프] EVA 30CM 마사지 폼롤러 (5900/무료)',
 '[위메프] LG 미니빔 HF60LA FHD 1400루멘 (835,640/무료)',
 '[11번가] 리스테린 그린티 마일드 750mlx2+250mlx2  (13,500/무료배송)',
 '[Gsuite] Gsuite Basic India (3700원(월) / 무료)',
 '[하이마트몰] 갤럭시버즈 (138,500원 / 무배) 오너스 첫달 무료체험시, 최대 50% 페이백',
 '[옥션] 식사에반하다25개입 (26,530원 / 무배)',
 '[티몬] (티몬페이전용)해피머니 10만원권 3장(271,500원/쿠폰적용시 269,900원)',
 '[티몬] 해피머니 10만원권 (90,900원/0원)',
 '[티몬] 1등급 한돈 삼겹살 600g (6,500/무료배송)',
 '[우체국쇼핑] 진안 매콤화끈 중독의맛 뼈없는 직화불닭발 1+1(2팩)(8500/무료) 쿠폰적용시7650',
 '[유튜브] 유튜브프리미엄 인도 ( 2200원(월), 3200원(월)6명 / 무료)',
 '[11번가] 벨킨 스마트폰 전기종 호환 충전케이블(9,900/무배)',
 '[네이버스토어팜] 뉴발란스 잔테v3 (49,800원/무료)',
 '[LG생활건강] 홈스타 뿌리는 곰팡이 (9,900/무료)']

In [140]:
for tr in dom.select("tr[class^=list]")[1:]:
    td = tr.find_all("td", recursive=False)
    print(requests.compat.urljoin(url, td[3].a.img["src"]))
    print(td[3].select("td[valign=middle] > a")[0].text)
    print(re,findall(".+\((.+?)\).*", temp)[0].split("/"))
    temp = td[5].text.split("-")
    print((0,0) if len(temp) < 2 else ",".join(temp))
    print(td[6].text.split())
    print()

AttributeError: 'NoneType' object has no attribute 'img'

# 실습
## 뽐뿌 자유게시판 Crawling
- Focused Crawling(10페이지 내에서)
- 각 페이지 당 게시글(제목) 부분의 a 수집
- 수집된 a로부터 Scraping
## scraping
- 제목, 본문내용, 댓글 파싱 (댓글은 있거나 없거나)

In [190]:
def parseURL(seed):
    html = download("get", seed)
    dom = BeautifulSoup(html.text, 'lxml')
    
    return [requests.compat.urljoin(seed, _.find_parent()["href"])
     for _ in dom.select("font.list_title")
            if _.find_parent() and _.find_parent().has_attr("href")]

In [201]:
def parseContent(url):
    html = download("get", url)
    dom = BeautifulSoup(html.text, 'lxml')
    
    return {"title":dom.select_one("font.view_title2").text.strip(),
           "body":dom.select_one("td.board-contents").text.strip(),
           "comments":[_.text.strip() for _ in dom.select("div[id^=commentContent_]")]}

In [210]:
url = "http://www.ppomppu.co.kr/zboard/zboard.php"
html = download("get", url, param = {"id":"freeboard"})
dom = BeautifulSoup(html.text, "lxml")

seed = list()
seed.extend([requests.compat.urljoin(url, _["href"]) for _ in dom.select("#page_list a")])
seen = list()

queue = list()

while seed:
    baseURL = seed.pop(0)
    seen.append(baseURL)
    linkList = parseURL(baseURL)
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue: {0}, Seen : {1}".format(len(queue), len(seen)))

Queue: 30, Seen : 1
Queue: 60, Seen : 2
Queue: 90, Seen : 3
Queue: 120, Seen : 4
Queue: 150, Seen : 5
Queue: 180, Seen : 6
Queue: 210, Seen : 7
Queue: 240, Seen : 8
Queue: 270, Seen : 9
Queue: 300, Seen : 10


In [203]:
# 확인하는 용도. 코드 실행 하지 마라. 아래꺼 안된다.
contents = list()
while queue:
    baseURL = queue.pop(0)
    seen.append(baseURL)
    contents.append(parseContent(baseURL))

In [204]:
len(contents)

300

In [195]:
# 참고 코드. 이해만 할 것.
for tr in dom.select("tr[class^=list]")[1:]:
    td = tr.find_all("td", recursive=False)
    print(requests.compat.urljoin(url, td[2].a["href"]))

http://www.ppomppu.co.kr/zboard/view.php?id=issue&no=160987
http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543291
http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543290
http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543289
http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543288
http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543287
http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543285
http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543284
http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543283
http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543281
http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543280
http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage

In [150]:
import sqlite3

con = sqlite3.connect("bot.db")
cur = con.cursor()

In [151]:
cur.executescript("""

    DROP TABLE IF EXISTS table3;
    CREATE TABLE table3(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        title TEXT NOT NULL,
        content TEXT NOT NULL,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
    
    DROP TABLE IF EXISTS table4;
    CREATE TABLE table4(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        table3_id INTEGER NOT NULL,
        comment TEXT NOT NULL
    );
""")
con.commit()

In [211]:
contents = list()
while queue:
    baseURL = queue.pop(0)
    seen.append(baseURL)
    content = parseContent(baseURL)
    cur.execute("""
        INSERT INTO table3(title, content) values (?, ?)
    """, [content["title"], content["body"]])
    con.commit()
    
    cur.execute("""
        select id
        from table3
        where title=? and content=?;
    """, [content["title"], content["body"]])
    table3_id = cur.fetchone()[0]
    
    for comment in content["comments"]:
        cur.execute("""
            insert into table4(table3_id, comment) values(?, ?)
        """, [table3_id, comment])
    con.commit()
    #contents.appennd(parseContent(baseURL))